# Data manipulation on CMS dataset
## Introduction
The goals of this notebook is for the user to understand:
- the roles of different kinds of ICD/NDC codes lists and how to use them to filter the raw CMS dataset.
- how to create the resulting dataset used for training a classification model
- incorporate these ideas to a different insurance database such as Optum




In [1]:
# but first we must know the dataset formats of the CMS.
import pandas as pd


inpatient = pd.read_csv('../datasets/cms_download/DE1_0_2008_to_2010_Inpatient_Claims_Sample_1.csv')
summary = pd.read_csv('../datasets/cms_download/DE1_0_2008_Beneficiary_Summary_File_Sample_1.csv')
prescriptions = pd.read_csv('../datasets/cms_download/DE1_0_2008_to_2010_Prescription_Drug_Events_Sample_1.csv')


/tmp/ipykernel_26809/453762595.py:7: DtypeWarning: Columns (3) have mixed types. Specify dtype option on import or set low_memory=False.
  prescriptions = pd.read_csv('../datasets/cms_download/DE1_0_2008_to_2010_Prescription_Drug_Events_Sample_1.csv')


In [2]:
pd.set_option('display.max_columns', None)


In [3]:
inpatient.head(5)

,DESYNPUF_ID,CLM_ID,SEGMENT,CLM_FROM_DT,CLM_THRU_DT,PRVDR_NUM,CLM_PMT_AMT,NCH_PRMRY_PYR_CLM_PD_AMT,AT_PHYSN_NPI,OP_PHYSN_NPI,OT_PHYSN_NPI,CLM_ADMSN_DT,ADMTNG_ICD9_DGNS_CD,CLM_PASS_THRU_PER_DIEM_AMT,NCH_BENE_IP_DDCTBL_AMT,NCH_BENE_PTA_COINSRNC_LBLTY_AM,NCH_BENE_BLOOD_DDCTBL_LBLTY_AM,CLM_UTLZTN_DAY_CNT,NCH_BENE_DSCHRG_DT,CLM_DRG_CD,ICD9_DGNS_CD_1,ICD9_DGNS_CD_2,ICD9_DGNS_CD_3,ICD9_DGNS_CD_4,ICD9_DGNS_CD_5,ICD9_DGNS_CD_6,ICD9_DGNS_CD_7,ICD9_DGNS_CD_8,ICD9_DGNS_CD_9,ICD9_DGNS_CD_10,ICD9_PRCDR_CD_1,ICD9_PRCDR_CD_2,ICD9_PRCDR_CD_3,ICD9_PRCDR_CD_4,ICD9_PRCDR_CD_5,ICD9_PRCDR_CD_6,HCPCS_CD_1,HCPCS_CD_2,HCPCS_CD_3,HCPCS_CD_4,HCPCS_CD_5,HCPCS_CD_6,HCPCS_CD_7,HCPCS_CD_8,HCPCS_CD_9,HCPCS_CD_10,HCPCS_CD_11,HCPCS_CD_12,HCPCS_CD_13,HCPCS_CD_14,HCPCS_CD_15,HCPCS_CD_16,HCPCS_CD_17,HCPCS_CD_18,HCPCS_CD_19,HCPCS_CD_20,HCPCS_CD_21,HCPCS_CD_22,HCPCS_CD_23,HCPCS_CD_24,HCPCS_CD_25,HCPCS_CD_26,HCPCS_CD_27,HCPCS_CD_28,HCPCS_CD_29,HCPCS_CD_30,HCPCS_CD_31,HCPCS_CD_32,HCPCS_CD_33,HCPCS_CD_34,HCPCS_CD_35,HCPCS_CD_36,HCPCS_CD_37,HCPCS_CD_38,HCPCS_CD_39,HCPCS_CD_40,HCPCS_CD_41,HCPCS_CD_42,HCPCS_CD_43,HCPCS_CD_44,HCPCS_CD_45
0,00013D2EFD8E45D1,196661176988405,1,20100312.0,20100313.0,2600GD,4000.0,0.0,3.139084e+09,NaN,NaN,20100312,4580,0.0,1100.0,0.0,0.0,1.0,20100313,217,7802,78820,V4501,4280,2720,4019,V4502,73300,E9330,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,00016F745862898F,196201177000368,1,20090412.0,20090418.0,3900MB,26000.0,0.0,6.476809e+09,NaN,NaN,20090412,7866,0.0,1068.0,0.0,0.0,6.0,20090418,201,1970,4019,5853,7843,2768,71590,2724,19889,5849,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,00016F745862898F,196661177015632,1,20090831.0,20090902.0,3900HM,5000.0,0.0,6.119985e+08,6.119985e+08,NaN,20090831,6186,0.0,1068.0,0.0,0.0,2.0,20090902,750,6186,2948,56400,NaN,NaN,NaN,NaN,NaN,NaN,NaN,7092.0,6186,V5866,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,00016F745862898F,196091176981058,1,20090917.0,20090920.0,3913XU,5000.0,0.0,4.971603e+09,NaN,1.119000e+09,20090917,29590,0.0,1068.0,0.0,0.0,3.0,20090920,883,29623,30390,71690,34590,V1581,32723,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,00016F745862898F,196261176983265,1,20100626.0,20100701.0,3900MB,16000.0,0.0,6.408400e+09,1.960860e+09,NaN,20100626,5849,0.0,1100.0,0.0,0.0,5.0,20100701,983,3569,4019,3542,V8801,78820,2639,7840,7856,4271,NaN,NaN,E8889,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [4]:
summary.head(5)

,DESYNPUF_ID,BENE_BIRTH_DT,BENE_DEATH_DT,BENE_SEX_IDENT_CD,BENE_RACE_CD,BENE_ESRD_IND,SP_STATE_CODE,BENE_COUNTY_CD,BENE_HI_CVRAGE_TOT_MONS,BENE_SMI_CVRAGE_TOT_MONS,BENE_HMO_CVRAGE_TOT_MONS,PLAN_CVRG_MOS_NUM,SP_ALZHDMTA,SP_CHF,SP_CHRNKIDN,SP_CNCR,SP_COPD,SP_DEPRESSN,SP_DIABETES,SP_ISCHMCHT,SP_OSTEOPRS,SP_RA_OA,SP_STRKETIA,MEDREIMB_IP,BENRES_IP,PPPYMT_IP,MEDREIMB_OP,BENRES_OP,PPPYMT_OP,MEDREIMB_CAR,BENRES_CAR,PPPYMT_CAR
0,00013D2EFD8E45D1,19230501,NaN,1,1,0,26,950,12,12,12,12,2,2,2,2,2,2,2,2,2,2,2,0.0,0.0,0.0,50.0,10.0,0.0,0.0,0.0,0.0
1,00016F745862898F,19430101,NaN,1,1,0,39,230,12,12,0,0,2,2,2,2,2,2,2,2,2,2,2,0.0,0.0,0.0,0.0,0.0,0.0,700.0,240.0,0.0
2,0001FDD721E223DC,19360901,NaN,2,1,0,39,280,12,12,0,12,2,2,2,2,2,2,2,2,2,2,2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,00021CA6FF03E670,19410601,NaN,1,5,0,6,290,0,0,0,0,2,2,2,2,2,2,2,2,2,2,2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,00024B3D2352D2D0,19360801,NaN,1,1,0,52,590,12,12,0,0,2,2,2,2,2,2,2,2,1,2,2,0.0,0.0,0.0,30.0,40.0,0.0,220.0,80.0,0.0


In [5]:
prescriptions.head(5)

,DESYNPUF_ID,PDE_ID,SRVC_DT,PROD_SRVC_ID,QTY_DSPNSD_NUM,DAYS_SUPLY_NUM,PTNT_PAY_AMT,TOT_RX_CST_AMT
0,00013D2EFD8E45D1,233664490397622,20080103,247037252,30.0,20,10.0,120.0
1,00013D2EFD8E45D1,233644490171972,20080105,223039502,10.0,10,0.0,0.0
2,00013D2EFD8E45D1,233974489116848,20080109,364724812,120.0,30,10.0,110.0
3,00013D2EFD8E45D1,233574491083209,20080123,179180672,30.0,30,0.0,240.0
4,00013D2EFD8E45D1,233024491180571,20080124,58016005300,30.0,30,70.0,70.0


while all files contain an ID column, prescriptions contains NDC codes, and inpatient contains ICD codes.

inpatient has multiple ICD columns while prescriptions contains just one relevant column for NDC. Thus differing filtering methods are going to be used.

### NDC, ICD Codes

This folder, `cms_tutorial/`, contains the necessary files for filtering the CMS dataset.

In [6]:
!ls *.txt

icd_cancer_regex_grepcode.txt  icd_opioid_harm_grepcode.txt
icd_cancer_regex_linecode.txt  icd_opioid_harm_linecode.txt
icd_hi_incidence_linecode.txt  ndc_filter_list_no_star_linecode.txt


In [7]:
# do notice that ndc_filter_list_no_star_linecode.txt conatains thousands of entries. Keeping this in the python file may increase the file size and slow down the IDE quite a bit. We can mitigate this by keeping this as a pickle file: 
import pickle

with open('filter_list_no_star', 'rb') as f:
    filter_list_no_star = pickle.load(f)


# dump a list as a pickle file using the following code:
# with open('filter_list_no_star', 'wb') as f:
#     pickle.dump(filter_list_no_star,f)
filter_list_no_star[10], len(filter_list_no_star)

('000020Y57', 41520)

### Query with list


In [8]:
# filter the prescriptions file for those prescribed opioids:
# access the list within the query term using @
# == @list : asks if the column value is included in the provided list. 

# According to the PDF describing the features in prescriptions, PROD_SRVC_ID is the column with NDC codes. We will filter out using this column.
presc_opioid_dframe = prescriptions.query('PROD_SRVC_ID == @filter_list_no_star').reset_index(drop = True)
presc_opioid_dframe

,DESYNPUF_ID,PDE_ID,SRVC_DT,PROD_SRVC_ID,QTY_DSPNSD_NUM,DAYS_SUPLY_NUM,PTNT_PAY_AMT,TOT_RX_CST_AMT
0,26D48234B1586158,233774488789326,20091217,00143100552,100.0,20,0.0,20.0
1,26D48234B1586158,233464492513148,20091220,63874020201,60.0,20,0.0,30.0
2,26D5584EEDAAE238,233704488950222,20090114,68115039799,20.0,80,0.0,20.0
3,26D5584EEDAAE238,233834489985005,20090725,33261005830,60.0,10,0.0,90.0
4,26D59CF00FF689CA,233144490974904,20091209,68462019301,120.0,10,10.0,10.0
...,...,...,...,...,...,...,...,...
28049,FDEBD2245BE84267,233024490882465,20091119,55289064830,60.0,20,10.0,170.0
28050,FDEDCC5E1626117B,233404489837851,20080509,63874020305,50.0,30,0.0,10.0
28051,FDEDCC5E1626117B,233934488876345,20080525,00591321372,30.0,30,0.0,70.0
28052,FDEDCC5E1626117B,233134492585708,20100415,58016007705,120.0,30,10.0,20.0


In [9]:
# compare with original prescriptions, substantial difference:
len(prescriptions)

5552421

In [10]:
# convert to list of IDs to filter in rest of files:
presc_opioid_list = list(set(presc_opioid_dframe.DESYNPUF_ID))

# inpatients,summary after first pass(filter opioid prescription)
inpatient_1 = inpatient.query('DESYNPUF_ID ==@presc_opioid_list').reset_index(drop=True)
summary_1 = summary.query('DESYNPUF_ID ==@presc_opioid_list').reset_index(drop= True)



In [11]:
# here is an example of how to filter using grep:
import os
def run_cmd_cancer_patients(n):
    '''
    
    given an input file, use grep on the inpatient claims to search for rows that include cancer diagnoses. convert this grep search into a dataframe csv


    Notes: there are many ways to improve upon this. 
    1. the output file could merely be line of ID codes. But implementing this will depend on the dataset format.
    2. the input file could be generalized. This could be helpful when you are not certain of the folder structure in the final product.
    3. at least for CMS, the cancer diagnosis flag is already is in the summary file. Since it is probably not the case for Optum, this code snippet may be useful.

    Also do note that grepping the inverse (i.e. include a line that does NOT contain the given search term) using a -v flag will merely include claims lines that do not include cancer diagnosis. The resulting list of patients will likely not decrease by much as they may have other diagnoses from a different day. For patient claims, we will need to load this new output as a dataframe to get the list of patients to filter out of the raw dataset.

    It is indeed possible to use the -v flag to filter out a given summary file, as each row contains a unique patient.
    '''
    # use sed to get the first line of the input file (for column names), and then run grep to get the search terms
    cmd = '''
    sed -n "1w ./cancer_out/cancer_patients%s.csv"  ../datasets/cms_download/DE1_0_2008_to_2010_Inpatient_Claims_Sample_%s.csv
    grep -E ',140..,|,141..,|,142..,|,143..,|,144..,|,145..,|,146..,|,147..,|,148..,|,149..,|,150..,|,151..,|,152..,|,153..,|,154..,|,155..,|,156..,|,156..,|,157..,|,158..,|,159..,|,160..,|,161..,|,162..,|,163..,|,164..,|,165..,|,170..,|,171..,|,172..,|,174..,|,1750.,|,1759.,|,176..,|,179,|,180..,|,181,|,182..,|,183..,|,184..,|,185,|,1860.,|,1869.,|,187..,|,188..,|,189..,|,190..,|,191..,|,192..,|,193,|,194..,|,195..,|,196..,|,197..,|,198..,|,19881,|,19882,|,19889,|,199..,|,2000.,|,2001.,|,2002.,|,2003.,|,2004.,|,2005.,|,2006.,|,2007.,|,2008.,|,2010.,|,2011.,|,2012.,|,2014.,|,2015.,|,2016.,|,2017.,|,2019.,|,2020.,|,2021.,|,2022.,|,2023.,|,2024.,|,2025.,|,2026.,|,2027.,|,2028.,|,2029.,|,2030.,|,2031.,|,2038.,|,2040.,|,2041.,|,2042.,|,2048.,|,2049.,|,2050.,|,2051.,|,2052.,|,2053.,|,2058.,|,2059.,|,2060.,|,2061.,|,2062.,|,2068.,|,2069.,|,2070.,|,2071.,|,2072.,|,2078.,|,2080.,|,2081.,|,2082.,|,2088.,|,2089.,|,2090.,|,2091.,|,2092.,|,2093.,|,2097.,' ../datasets/cms_download/DE1_0_2008_to_2010_Inpatient_Claims_Sample_%s.csv >> ./cancer_out/cancer_patients%s.csv
    ''' %(n,n,n,n)
    os.system(cmd)
    os.system('echo "run_cmd_cancer_patients: accessed unifp_op%s to create ./cancer_out/cancer_patients%s.csv"'%(n,n))
run_cmd_cancer_patients(1)  


run_cmd_cancer_patients: accessed unifp_op1 to create ./cancer_out/cancer_patients1.csv


run_cmd_opioid_harm_patients: accessed op_prescribed_no_cancer_df1.csv to create opioid_harm_patients1.csv


sed: couldn't open file ./opioid_harm_out/opioid_harm_patients1.csv: No such file or directory
grep: ./op_prescribed_no_cancer_df1.csv: No such file or directory


In [13]:
cancer_patient_dframe = pd.read_csv('cancer_out/cancer_patients1.csv')
cancer_patient_dframe

,DESYNPUF_ID,CLM_ID,SEGMENT,CLM_FROM_DT,CLM_THRU_DT,PRVDR_NUM,CLM_PMT_AMT,NCH_PRMRY_PYR_CLM_PD_AMT,AT_PHYSN_NPI,OP_PHYSN_NPI,OT_PHYSN_NPI,CLM_ADMSN_DT,ADMTNG_ICD9_DGNS_CD,CLM_PASS_THRU_PER_DIEM_AMT,NCH_BENE_IP_DDCTBL_AMT,NCH_BENE_PTA_COINSRNC_LBLTY_AM,NCH_BENE_BLOOD_DDCTBL_LBLTY_AM,CLM_UTLZTN_DAY_CNT,NCH_BENE_DSCHRG_DT,CLM_DRG_CD,ICD9_DGNS_CD_1,ICD9_DGNS_CD_2,ICD9_DGNS_CD_3,ICD9_DGNS_CD_4,ICD9_DGNS_CD_5,ICD9_DGNS_CD_6,ICD9_DGNS_CD_7,ICD9_DGNS_CD_8,ICD9_DGNS_CD_9,ICD9_DGNS_CD_10,ICD9_PRCDR_CD_1,ICD9_PRCDR_CD_2,ICD9_PRCDR_CD_3,ICD9_PRCDR_CD_4,ICD9_PRCDR_CD_5,ICD9_PRCDR_CD_6,HCPCS_CD_1,HCPCS_CD_2,HCPCS_CD_3,HCPCS_CD_4,HCPCS_CD_5,HCPCS_CD_6,HCPCS_CD_7,HCPCS_CD_8,HCPCS_CD_9,HCPCS_CD_10,HCPCS_CD_11,HCPCS_CD_12,HCPCS_CD_13,HCPCS_CD_14,HCPCS_CD_15,HCPCS_CD_16,HCPCS_CD_17,HCPCS_CD_18,HCPCS_CD_19,HCPCS_CD_20,HCPCS_CD_21,HCPCS_CD_22,HCPCS_CD_23,HCPCS_CD_24,HCPCS_CD_25,HCPCS_CD_26,HCPCS_CD_27,HCPCS_CD_28,HCPCS_CD_29,HCPCS_CD_30,HCPCS_CD_31,HCPCS_CD_32,HCPCS_CD_33,HCPCS_CD_34,HCPCS_CD_35,HCPCS_CD_36,HCPCS_CD_37,HCPCS_CD_38,HCPCS_CD_39,HCPCS_CD_40,HCPCS_CD_41,HCPCS_CD_42,HCPCS_CD_43,HCPCS_CD_44,HCPCS_CD_45
0,00016F745862898F,196201177000368,1,20090412,20090418,3900MB,26000.0,0.0,6.476809e+09,NaN,NaN,20090412,7866,0.0,1068.0,0.0,0.0,6,20090418,201,1970,4019,5853,7843,2768,71590,2724,19889,5849,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,0021B3C854C968C8,196711176994160,1,20081208,20081213,1700JJ,5000.0,0.0,4.712447e+09,8.184097e+09,NaN,20081208,27651,0.0,1024.0,0.0,0.0,5,20081213,615,2768,60000,20300,25000,5718,V451,2930,5856,V1005,NaN,3895.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,0021D4CDAFC0609F,196981177001394,1,20090428,20090503,3302XJ,5000.0,0.0,2.215254e+09,NaN,2.215254e+09,20090428,78605,0.0,1068.0,0.0,0.0,5,20090503,206,49122,4439,20480,72610,V5863,41401,60000,3051,25000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,003A7E32FEBE9F4D,196021176981603,1,20091123,20091130,5001PC,11000.0,0.0,1.776996e+09,NaN,8.045969e+09,20091123,5781,0.0,1068.0,0.0,0.0,7,20091130,368,5789,2449,5854,28860,496,71590,53540,78605,1977,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,003D67A9716DC59A,196551176976408,1,20080518,20080605,1920WH,8000.0,0.0,4.585434e+09,8.326600e+09,NaN,20080518,5789,0.0,1024.0,0.0,0.0,18,20080605,385,56212,2859,70724,496,7906,0389,42789,5990,5856,NaN,9904.0,2867,27651,53019,1985,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4448,FFB5A3FBC31ABA7E,196111177004698,1,20100928,20101002,35S0NC,20000.0,0.0,2.434265e+09,NaN,NaN,20100928,2859,0.0,1100.0,0.0,0.0,4,20101002,842,20500,6821,29630,60886,5781,4553,20410,28800,4414,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4449,FFB66E6F338A1C5C,196051177020464,1,20080713,20080721,05T3KM,7000.0,0.0,9.738053e+09,4.137692e+09,NaN,200807

In [14]:
# extract the list of patients from the resulting dframe that were diagnosed with cancer:
cancer_patient_list = list(set(cancer_patient_dframe.DESYNPUF_ID))
cancer_patient_list

['42A62EC83635CA1F',
 'D2E943EA83FA9025',
 '8B4BD31C5EA3BD1C',
 '1ABAA2D14AECE7DA',
 '10D754471B9ECC3C',
 'D4477974A052E756',
 'E17371BA6B71A68D',
 '40F295CC76D0AFF2',
 '34AC0045D00EAA6E',
 'E111C2888C43B3B8',
 'FC52CEBDED1077B1',
 '10C7B8099BC79C16',
 '1A2597A7F6871BA7',
 '6F455D8260C4D62E',
 'F82EAFC89B510FB5',
 '4F4E01E9765A48B6',
 '28EA848413E8B212',
 'CB7AB0153706C908',
 'FD2CE91D7D07247E',
 '315209705BC820A9',
 '0A3EC832A15F4736',
 '7A8D6420C101B237',
 '1EC1A05FDBED5F32',
 'BFC2517DA974771A',
 '8906A25F608FC079',
 'C370BD5E920D9334',
 '043E9B9F2E5CD779',
 '4115FBE64850AA4C',
 '42EB4652A7A6E1D3',
 'B9EE622720A19531',
 'CAC5BAA1B371A28E',
 'D4AD11DD1560D359',
 '1C6C04B4EC38CEDA',
 'D870E84200C747CD',
 'F24BE5436BE2FEBD',
 '39A91E77F42B2CEF',
 '4B4C1B557BC1EEF3',
 '501B34536C8AE668',
 '7DD13E1901664BA6',
 '231D41CF60C316A5',
 'C0B57BB8D8BAC6DF',
 '3E7192606E5B2351',
 '5858672194313C14',
 'E4E6A97CBFAF41A0',
 '892470DDB3CEEB89',
 'E3152528F2F8FE35',
 '5F49B18EE0AEF93B',
 '47FDA6CE6FF

In [15]:
# filter out the list of patients with cancer diagnosis:
# access the list within the query term using @
# != @list : asks if the column value is not included in the provided list. 
inpatient_2 = inpatient_1.query('DESYNPUF_ID !=@cancer_patient_list').reset_index(drop= True)
summary_2 = summary_1.query('DESYNPUF_ID !=@cancer_patient_list').reset_index(drop=True)
# inpatient_2
summary_2

,DESYNPUF_ID,BENE_BIRTH_DT,BENE_DEATH_DT,BENE_SEX_IDENT_CD,BENE_RACE_CD,BENE_ESRD_IND,SP_STATE_CODE,BENE_COUNTY_CD,BENE_HI_CVRAGE_TOT_MONS,BENE_SMI_CVRAGE_TOT_MONS,BENE_HMO_CVRAGE_TOT_MONS,PLAN_CVRG_MOS_NUM,SP_ALZHDMTA,SP_CHF,SP_CHRNKIDN,SP_CNCR,SP_COPD,SP_DEPRESSN,SP_DIABETES,SP_ISCHMCHT,SP_OSTEOPRS,SP_RA_OA,SP_STRKETIA,MEDREIMB_IP,BENRES_IP,PPPYMT_IP,MEDREIMB_OP,BENRES_OP,PPPYMT_OP,MEDREIMB_CAR,BENRES_CAR,PPPYMT_CAR
0,26D48234B1586158,19390501,NaN,2,1,0,51,180,12,12,0,12,1,1,1,2,2,1,1,1,1,2,2,0.0,0.0,0.0,1010.0,500.0,0.0,870.0,200.0,0.0
1,26D5584EEDAAE238,19370101,NaN,2,2,0,31,150,12,12,0,12,2,1,2,2,2,1,2,1,1,2,2,0.0,0.0,0.0,20.0,50.0,0.0,800.0,460.0,0.0
2,26D59CF00FF689CA,19200301,NaN,2,1,0,7,0,12,12,0,12,1,2,1,2,1,1,1,1,1,2,2,11200.0,1024.0,0.0,650.0,400.0,3000.0,1350.0,350.0,0.0
3,26D5A2B67FF2F770,19580901,NaN,1,1,0,10,120,12,12,0,12,2,2,2,2,2,2,2,1,2,2,2,0.0,0.0,0.0,170.0,340.0,0.0,290.0,40.0,80.0
4,26D7F5CD9BCC7ECB,19510201,NaN,1,1,0,34,230,3,3,0,0,2,1,2,2,2,2,2,1,2,2,2,0.0,0.0,0.0,0.0,0.0,0.0,1120.0,280.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10952,FDE61CA99097DABA,19240201,NaN,1,1,0,39,10,2,2,0,3,2,2,2,2,2,2,2,2,2,2,2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
10953,FDE9D86C44F4A51B,19330501,NaN,1,1,0,39,770,12,12,0,12,1,1,1,1,2,1,2,1,2,2,2,0.0,0.0,0.0,350.0,410.0,0.0,3260.0,1010.0,140.0
10954,FDEBD2245BE84267,19310601,NaN,1,1,0,45,910,12,12,0,12,1,2,2,2,2,1,1,1,2,2,2,0.0,0.0,0.0,0.0,0.0,0.0,970.0,220.0,0.0
10955,FDEDCC5E1626117B,19460501,NaN,2,1,Y,37,560,12,12,0,12,1,1,1,1,1,1,1,1,1,2,1,11900.0,3072.0,0.0,940.0,770.0,200.0,4790.0,1750.0,40.0


In [16]:

# inpatient_3 = 

### inpatient file into training data

If we were to get the outcome (opioid-related harm), we should run run_cmd_opioid_harm_patients() with the input,output locations added in. But I will omit this for this tutorial. 

In this tutorial, we assume that the user wants a dataset of patients who are prescribed with opioids and are not diagnosed with cancer.


In [ ]:
# here is the code snippet for run_cmd_opioid_harm_patients:
def run_cmd_opioid_harm_patients(n):
    # following icd9 codes correspond to opioid related harm 
    # source: supplementary appendix for developing prediction algorithm using Pennsylvania Medicaid data, jonassan et al.
    cmd = '''
    sed -n "1w ./opioid_harm_out/opioid_harm_patients%s.csv"  ./op_prescribed_no_cancer_df%s.csv
    grep -E ',96500,|,96501,|,96502,|,96509,|,E8500,|,E8501,|,E8502,|,E9350,|,E9351,|,E9352,|,3055,|,3040,|,3047,|,E9350,|,E9351,|,E9352,|,E9401,|,9701,' ./op_prescribed_no_cancer_df%s.csv >> opioid_harm_patients%s.csv
    '''%(n,n,n,n)
    os.system(cmd)
    os.system('echo "run_cmd_opioid_harm_patients: accessed op_prescribed_no_cancer_df%s.csv to create opioid_harm_patients%s.csv"'%(n,n))
# run_cmd_opioid_harm_patients(1)

In [17]:
# The following code was used with inpatient/outpatient joined. but using them without joining also works.

# Note to self: this code could be used in place of grep to label patients experiencing opioid harm. But it likely runs slower due to the loop iterating through the dframe.

def label_icd(unif, codes):
    '''
    unif: inpatient+outpatient df which contains ICD9 columns 
    codes: names of ICD9 codes to check for each row
    add a new icd9 column for each code in codes labeling whether the patient had that diagnosis
    '''
    # for x in hi_inc_icd9.ICD_CODE:
    print(list(codes))
    for x in codes:
        # print(x)
        icd_str = 'ICD_'+str(x)
        unif[icd_str] = (unif.ICD9_DGNS_CD_1 == str(x) )|(unif.ICD9_DGNS_CD_2 == str(x) )|(unif.ICD9_DGNS_CD_3 == str(x) )|(unif.ICD9_DGNS_CD_4 == str(x) )|(unif.ICD9_DGNS_CD_5 == str(x) )|(unif.ICD9_DGNS_CD_6 == str(x) )|(unif.ICD9_DGNS_CD_7 == str(x) )|(unif.ICD9_DGNS_CD_8 == str(x) )|(unif.ICD9_DGNS_CD_9 == str(x) )|(unif.ICD9_DGNS_CD_10 == str(x) )
        # convert to binary true/false
        unif[icd_str] = unif[icd_str].astype(int)
    return unif


In [18]:
# from ../data analysis/analyze_harm.ipynb, comparing the icd incidence between those who experienced opioid harm and those who have not resulted in the following ICD codes.

icd_hi_incidence = [28521,496,58881,5990,2809,41401,3051,311,5859,5849,5856,27651,30000]

inpatient_hi_incidence_dframe = label_icd(inpatient_2, icd_hi_incidence)
inpatient_hi_incidence_dframe


[28521, 496, 58881, 5990, 2809, 41401, 3051, 311, 5859, 5849, 5856, 27651, 30000]


,DESYNPUF_ID,CLM_ID,SEGMENT,CLM_FROM_DT,CLM_THRU_DT,PRVDR_NUM,CLM_PMT_AMT,NCH_PRMRY_PYR_CLM_PD_AMT,AT_PHYSN_NPI,OP_PHYSN_NPI,OT_PHYSN_NPI,CLM_ADMSN_DT,ADMTNG_ICD9_DGNS_CD,CLM_PASS_THRU_PER_DIEM_AMT,NCH_BENE_IP_DDCTBL_AMT,NCH_BENE_PTA_COINSRNC_LBLTY_AM,NCH_BENE_BLOOD_DDCTBL_LBLTY_AM,CLM_UTLZTN_DAY_CNT,NCH_BENE_DSCHRG_DT,CLM_DRG_CD,ICD9_DGNS_CD_1,ICD9_DGNS_CD_2,ICD9_DGNS_CD_3,ICD9_DGNS_CD_4,ICD9_DGNS_CD_5,ICD9_DGNS_CD_6,ICD9_DGNS_CD_7,ICD9_DGNS_CD_8,ICD9_DGNS_CD_9,ICD9_DGNS_CD_10,ICD9_PRCDR_CD_1,ICD9_PRCDR_CD_2,ICD9_PRCDR_CD_3,ICD9_PRCDR_CD_4,ICD9_PRCDR_CD_5,ICD9_PRCDR_CD_6,HCPCS_CD_1,HCPCS_CD_2,HCPCS_CD_3,HCPCS_CD_4,HCPCS_CD_5,HCPCS_CD_6,HCPCS_CD_7,HCPCS_CD_8,HCPCS_CD_9,HCPCS_CD_10,HCPCS_CD_11,HCPCS_CD_12,HCPCS_CD_13,HCPCS_CD_14,HCPCS_CD_15,HCPCS_CD_16,HCPCS_CD_17,HCPCS_CD_18,HCPCS_CD_19,HCPCS_CD_20,HCPCS_CD_21,HCPCS_CD_22,HCPCS_CD_23,HCPCS_CD_24,HCPCS_CD_25,HCPCS_CD_26,HCPCS_CD_27,HCPCS_CD_28,HCPCS_CD_29,HCPCS_CD_30,HCPCS_CD_31,HCPCS_CD_32,HCPCS_CD_33,HCPCS_CD_34,HCPCS_CD_35,HCPCS_CD_36,HCPCS_CD_37,HCPCS_CD_38,HCPCS_CD_39,HCPCS_CD_40,HCPCS_CD_41,HCPCS_CD_42,HCPCS_CD_43,HCPCS_CD_44,HCPCS_CD_45,ICD_28521,ICD_496,ICD_58881,ICD_5990,ICD_2809,ICD_41401,ICD_3051,ICD_311,ICD_5859,ICD_5849,ICD_5856,ICD_27651,ICD_30000
0,26D5584EEDAAE238,196481176974876,1,20090903.0,20090906.0,3100XU,4000.0,0.0,7.726984e+09,NaN,NaN,20090903,78039,200.0,1068.0,0.0,0.0,3.0,20090906,614,2536,2449,25000,78079,2859,5589,30390,73300,V5861,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,0,0,0,0,0,0,0,0,0,0,0,0
1,26D59CF00FF689CA,196191177010852,1,20080814.0,20080816.0,1000AH,11000.0,0.0,8.891701e+09,5.112613e+09,NaN,20080814,5990,100.0,1024.0,0.0,0.0,2.0,20080816,463,72740,496,4019,0417,5990,2851,5849,4439,3004,NaN,8480.0,5859,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,1,0,1,0,0,0,0,0,1,0,0,0
2,26DAD637036537FE,196111177015442,1,20081209.0,20081210.0,0502TV,12000.0,0.0,5.797374e+09,9.567259e+09,NaN,20081209,41401,20.0,1024.0,0.0,0.0,1.0,20081210,281,41071,4111,42769,53010,2749,25000,NaN,NaN,NaN,NaN,66.0,4111,42731,NaN,4019,41401,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,0,0,0,0,0,0,0,0,0,0,0,0
3,26DAD637036537FE,196671177005624,1,20090227.0,20090304.0,0503SD,5000.0,0.0,1.323186e+09,NaN,NaN,20090227,78009,200.0,1068.0,0.0,0.0,5.0,20090304,883,29021,42731,7866,2859,2720,29590,2724,7830,30390,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,0,0,0,0,0,0,0,0,0,0,0,0
4,26DAD637036537FE,196411177001285,1,20090409.0,20090411.0,0500KV,9000.0,9000.0,9.938896e+09,1.477205e+09,NaN,20090409,99667,0.0,1068.0,0.0,0.0,2.0,20090411,463,73017,2449,2859,311,NaN,NaN,NaN,NaN,NaN,NaN,7867.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,0,0,0,0,0,0,1,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6906,FDEDCC5E1626117B,196181176977633,1,20080613.0,20080616.0,3700WD,5000.0,0.0,4.701152e+09,3.970200e+09,NaN,20080613,72981,0.0,1024.0,0.0,0.0,3.0,20080616,257,4280,4168,V4502,412,1539,42731,V

In [19]:
# take the aggregate of the labeled icd to count how many times a patient gets the icd code

# so within this inpatient file, we see how many times the patient was diagnosed a certain ICD code.
inpatient_hi_incidence_agg = inpatient_hi_incidence_dframe.groupby('DESYNPUF_ID').agg(sum).reset_index()
incidence_column_names = ['DESYNPUF_ID'] + ['ICD_' + str(x) for x in icd_hi_incidence]
inpatient_hi_incidence_tojoin = inpatient_hi_incidence_agg[incidence_column_names]
inpatient_hi_incidence_tojoin


/tmp/ipykernel_26809/3342322460.py:2: FutureWarning: The default value of numeric_only in DataFrameGroupBy.sum is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  inpatient_hi_incidence_agg = inpatient_hi_incidence_dframe.groupby('DESYNPUF_ID').agg(sum).reset_index()


,DESYNPUF_ID,ICD_28521,ICD_496,ICD_58881,ICD_5990,ICD_2809,ICD_41401,ICD_3051,ICD_311,ICD_5859,ICD_5849,ICD_5856,ICD_27651,ICD_30000
0,26D5584EEDAAE238,0,0,0,0,0,0,0,0,0,0,0,0,0
1,26D59CF00FF689CA,0,1,0,1,0,0,0,0,0,1,0,0,0
2,26DAD637036537FE,0,0,0,1,0,1,0,1,0,1,0,0,0
3,26DB9087A76F42AB,0,0,0,0,0,0,0,0,0,0,0,0,0
4,26DBA17C5D856F47,0,0,0,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3894,FDD9860D3D264813,0,0,0,0,0,0,0,0,0,0,0,0,0
3895,FDDCEAE8AC08BB83,0,1,0,0,0,0,0,0,0,0,0,0,0
3896,FDE4AF9F81E7695F,0,1,0,1,0,0,0,1,0,0,0,0,0
3897,FDEBD2245BE84267,0,0,0,0,0,0,0,0,0,0,0,0,0


In [22]:
# Set the summary file to have the same list of patients as the filtered inpatient file.
inpatient_list = list(set(inpatient_2.DESYNPUF_ID))
summary_3 = summary_2.query('DESYNPUF_ID == @inpatient_list').reset_index(drop= True)

In [24]:
dataset_presc_noncancer = summary_3.join(inpatient_hi_incidence_tojoin.set_index('DESYNPUF_ID'),on='DESYNPUF_ID')

dataset_presc_noncancer

,DESYNPUF_ID,BENE_BIRTH_DT,BENE_DEATH_DT,BENE_SEX_IDENT_CD,BENE_RACE_CD,BENE_ESRD_IND,SP_STATE_CODE,BENE_COUNTY_CD,BENE_HI_CVRAGE_TOT_MONS,BENE_SMI_CVRAGE_TOT_MONS,BENE_HMO_CVRAGE_TOT_MONS,PLAN_CVRG_MOS_NUM,SP_ALZHDMTA,SP_CHF,SP_CHRNKIDN,SP_CNCR,SP_COPD,SP_DEPRESSN,SP_DIABETES,SP_ISCHMCHT,SP_OSTEOPRS,SP_RA_OA,SP_STRKETIA,MEDREIMB_IP,BENRES_IP,PPPYMT_IP,MEDREIMB_OP,BENRES_OP,PPPYMT_OP,MEDREIMB_CAR,BENRES_CAR,PPPYMT_CAR,ICD_28521,ICD_496,ICD_58881,ICD_5990,ICD_2809,ICD_41401,ICD_3051,ICD_311,ICD_5859,ICD_5849,ICD_5856,ICD_27651,ICD_30000
0,26D5584EEDAAE238,19370101,NaN,2,2,0,31,150,12,12,0,12,2,1,2,2,2,1,2,1,1,2,2,0.0,0.0,0.0,20.0,50.0,0.0,800.0,460.0,0.0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,26D59CF00FF689CA,19200301,NaN,2,1,0,7,0,12,12,0,12,1,2,1,2,1,1,1,1,1,2,2,11200.0,1024.0,0.0,650.0,400.0,3000.0,1350.0,350.0,0.0,0,1,0,1,0,0,0,0,0,1,0,0,0
2,26DAD637036537FE,19350301,NaN,1,5,0,5,400,12,12,0,12,2,2,2,2,1,2,1,1,1,1,2,12020.0,1024.0,0.0,990.0,250.0,0.0,1130.0,250.0,100.0,0,0,0,1,0,1,0,1,0,1,0,0,0
3,26DB9087A76F42AB,19391101,NaN,2,1,0,14,141,12,8,0,12,1,2,2,2,2,2,2,2,1,2,1,4400.0,1024.0,0.0,200.0,310.0,0.0,930.0,220.0,0.0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,26DBA17C5D856F47,19290201,NaN,2,1,0,42,220,12,12,0,12,2,2,2,2,2,1,2,1,1,2,2,0.0,0.0,0.0,1810.0,910.0,0.0,1480.0,530.0,0.0,0,0,0,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3894,FDD9860D3D264813,19290501,NaN,2,1,Y,4,590,12,12,12,12,1,1,1,2,1,2,1,1,1,2,1,0.0,0.0,0.0,15430.0,2650.0,1000.0,8150.0,2400.0,0.0,0,0,0,0,0,0,0,0,0,0,0,0,0
3895,FDDCEAE8AC08BB83,19421101,NaN,2,1,0,14,992,0,0,0,12,2,2,2,2,2,1,2,1,2,2,2,0.0,0.0,0.0,520.0,80.0,0.0,30.0,0.0,0.0,0,1,0,0,0,0,0,0,0,0,0,0,0
3896,FDE4AF9F81E7695F,19420601,NaN,1,1,0,50,260,12,12,12,11,1,1,2,2,1,1,2,1,2,2,2,4250.0,1024.0,0.0,0.0,0.0,0.0,100.0,70.0,0.0,0,1,0,1,0,0,0,1,0,0,0,0,0
3897,FDEBD2245BE84267,19310601,NaN,1,1,0,45,910,12,12,0,12,1,2,2,2,2,1,1,1,2,2,2,0.0,0.0,0.0,0.0,0.0,0.0,970.0,220.0,0.0,0,0,0,0,0,0,0,0,0,0,0,0,0


### Conclusion

This marks the end of the tutorial. The possible next steps:
1. have expert opinion, instead of ICD code prevalence comparison, for additional features
2. have dosage measurements, using the prescriptions file to get the MME for the patient's prescription opioid
3. early refill count, using prescriptions file days_supply column

Hopefully this tutorial helps visualize the applications for the implementation in both CMS and Optum.